<a href="https://colab.research.google.com/github/mohmadAyman75/Face-classification/blob/main/only.5.classes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
upload = files.upload()

In [ ]:
import os
import cv2
import numpy as np
import zipfile
import tempfile
ZIP_PATH = "Aligned_Faces.zip"
IMG_SIZE = 224
TARGET_IMAGES = 20
temp_dir = tempfile.mkdtemp()
with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
    zip_ref.extractall(temp_dir)
DATASET_DIR = temp_dir
if os.path.isdir(os.path.join(DATASET_DIR, 'Aligned_Faces')):
    DATASET_DIR = os.path.join(DATASET_DIR, 'Aligned_Faces')
contents = [d for d in os.listdir(DATASET_DIR) if os.path.isdir(os.path.join(DATASET_DIR, d))]
if len(contents) == 1 and contents[0] == 'Aligned_Faces':
    DATASET_DIR = os.path.join(DATASET_DIR, 'Aligned_Faces')

print("Dataset extracted to", DATASET_DIR)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.10,
    height_shift_range=0.10,
    zoom_range=0.15,
    brightness_range=[0.7, 1.3],
    horizontal_flip=True
)

def generate_augmented_images(image, save_dir, start_index, n):
    image = np.expand_dims(image, axis=0)
    gen = datagen.flow(image, batch_size=1)

    for i in range(n):
        aug = next(gen)[0].astype("uint8")
        out_path = os.path.join(save_dir, f"aug_{start_index + i}.jpg")
        cv2.imwrite(out_path, aug)


In [ ]:
persons = os.listdir(DATASET_DIR)
persons = persons[:5]

for person in persons:
    pf = os.path.join(DATASET_DIR, person)
    if not os.path.isdir(pf):
        continue

    imgs = [f for f in os.listdir(pf) if f.lower().endswith((".jpg", ".png"))]

    count = len(imgs)
    print(f"[Processing] {person} → {count}")

    if count == 0:
        print(f"No images found for {person}. Skipping augmentation.\n")
        continue

    if count >= TARGET_IMAGES:
        continue

    missing = TARGET_IMAGES - count

    base_img = cv2.imread(os.path.join(pf, imgs[0]))
    base_img = cv2.resize(base_img, (IMG_SIZE, IMG_SIZE))

    generate_augmented_images(base_img, pf, count, missing)
    print(f"Added {missing} images\n")

print("\nAugmentation Done")


[Processing] Alan_Dreher → 20
[Processing] Barbara_Bach → 20
[Processing] Ilham_Aliev → 20
[Processing] Jeane_Kirkpatrick → 20
[Processing] Stefan_Tafrov → 20

Augmentation Done


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import os

images = []
labels = []
persons = os.listdir(DATASET_DIR)
persons = persons[:5]

for person in persons:
    pf = os.path.join(DATASET_DIR, person)
    if not os.path.isdir(pf):
        continue

    for f in os.listdir(pf):
        if f.lower().endswith((".jpg", ".png", ".jpeg")):
            path = os.path.join(pf, f)
            img = cv2.imread(path)
            if img is None:
                continue

            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))

            images.append(img)
            labels.append(person)

images = np.array(images)
labels = np.array(labels)

print("Images:", images.shape)
print("Labels:", labels.shape)


In [ ]:
le = LabelEncoder()
labels_encoded = le.fit_transform(labels)

X_train, X_test, y_train, y_test = train_test_split(
    images, labels_encoded,
    test_size=0.2,
    stratify=labels_encoded,
    random_state=42
)

print("Train:", X_train.shape)
print("Test:", X_test.shape)


In [ ]:
X_train = X_train.astype("float32") / 255.0
X_test  = X_test.astype("float32") / 255.0
from tensorflow.keras.utils import to_categorical
num_classes = len(np.unique(labels_encoded))
y_train= to_categorical(y_train, num_classes)
y_test= to_categorical(y_test, num_classes)
print("OneHot Shapes:", y_train.shape, y_test.shape)

In [ ]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
base_model = EfficientNetB0(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
output = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(
    optimizer=Adam(learning_rate=0.0005),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


In [ ]:
history = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=16,
    validation_data=(X_test, y_test)
)


Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 30s 2s/step - accuracy: 0.0539 - loss: 3.0033 - val_accuracy: 0.0500 - val_loss: 2.9988
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 38s 1s/step - accuracy: 0.0357 - loss: 3.0272 - val_accuracy: 0.0750 - val_loss: 2.9969
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 26s 1s/step - accuracy: 0.0621 - loss: 3.0169 - val_accuracy: 0.0500 - val_loss: 2.9974
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 46s 2s/step - accuracy: 0.0206 - loss: 3.0433 - val_accuracy: 0.0500 - val_loss: 2.9982
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 31s 2s/step - accuracy: 0.0478 - loss: 3.0094 - val_accuracy: 0.0500 - val_loss: 2.9981
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 26s 1s/step - accuracy: 0.0349 - loss: 3.0102 - val_accuracy: 0.0500 - val_loss: 2.9995
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 40s 1s/step - accuracy: 0.0625 - loss: 3.0450 - val_accuracy: 0.0500 - val_loss: 2.9986
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 46s 2s/step - accuracy: 0.0400 - loss: 3.0060 - val_accuracy: 0.0500 - val_loss:

In [ ]:
loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc*100:.2f}%")
